# Data Analysis using Plotly

In this notebook, we take on a visual quanitative approach for exploring WACC (-parameter) of DAX-30 companies.

From a technical perspective, we make use of the plotly library that allows for creating interactive visualizations using python.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GitHub/00_Repo_Phd-Analysen/(1)_DAX_Explorative WACC_Analyse/WACC_der DAX-30_filled_missings_20220221_final.csv",
                 error_bad_lines = False,
                 sep = ";",
                 decimal = ",",
                 index_col = ["Jahr"])

df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version.





,Unternehmen,Sektor,Industrie,WACC_cMR,WACC_15yMR,WACC_10yMR,WACC_GB_nSt,WACC_GB_vSt,CAPM_cMR,CAPM_15yMR,CAPM_10yMR,Beta,risikoloser Zins,Bilanzsumme_BW,Eigenkapital_BW,Fremdkapital_BW,EK-Quote_BW,FK-Quote_BW,Gesamtkapital_MKT,Marktkapitalisierung,FK-Kosten_bereinigt,EK-Quote_MKT,FK-Quote_MKT,FK-Kosten_unbereinigt,Zinsaufwand,FK_zinstragend,Steuersaetze,Marktrendite_Mittelwert,Marktrendite_1Y,Marktrendite_10Y,Investitionsquote,Umsatzerloese
Jahr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0513,0.0472,0.0443,0.070,0.0898,0.0711,0.0585,0.04950,0.50,0.0410,4427.0,1628.0,2799.0,0.367743,0.632257,4154.0,1355.0,0.067074,0.326248,0.673752,0.025473,71.3,1063.0,0.378,0.08,0.073,0.073,0.000887,5860.0
2005.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0530,0.0448,0.0475,0.081,0.1008,0.0716,0.0500,0.05717,0.51,0.0407,5722.0,2684.0,3038.0,0.469067,0.530933,4908.0,1870.0,0.062606,0.380960,0.619040,0.024358,74.0,1182.0,0.337,0.08,0.271,0.091,-0.056687,6636.0
2006.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0618,0.0594,0.0584,0.075,0.0948,0.0729,0.0687,0.06698,0.58,0.0338,8372.0,2828.0,5544.0,0.337793,0.662207,13218.0,7674.0,0.067847,0.580569,0.419431,0.033189,184.0,2712.0,0.314,0.08,0.220,0.086,2.744601,10084.0
2007.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0435,0.0435,0.0432,0.075,0.0948,0.0397,0.0397,0.03925,0.03,0.0378,8314.0,3023.0,5291.0,0.363604,0.636396,15727.0,10436.0,0.074890,0.663573,0.336427,0.032130,170.0,2270.0,0.318,0.08,0.223,0.066,-0.065697,10299.0
2008.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0617,0.0651,0.0537,0.083,0.1028,0.0712,0.0784,0.05391,0.49,0.0423,9519.0,3386.0,6133.0,0.355710,0.644290,11495.0,5362.0,0.075046,0.466453,0.533547,0.033100,203.0,2705.0,0.288,0.08,-0.404,-0.004,0.093666,10799.0


In [ ]:
# Rename columns ###############################################################
wacc = df.rename(columns = {"Unternehmen": "Company",
                            "Sektor": "Sector",
                            "Industrie": "Industry",
                            " WACC_GB_nSt ": "WACC_FS_aT",
                            " WACC_GB_vSt ": "WACC_FS_bT",
                            "Eigenkapitalquote_MKT": "Equity_ratio_market",
                            " Fremdkapitalquote_MKT ": "Debt_ratio_market",
                            "risikoloser Zins": "Risk_free_interest",
                            "Bilanzsumme_BW": "Balance_sum",
                            "Eigenkapital_BW": "Equity_book",
                            "Fremdkapital_BW": "Debt_book",
                            "EK-Quote_BW": "Equity_ratio_book",
                            "FK-Quote_BW": "Debt_ratio_book",
                            "Gesamtkapital_MKT": "Total_assets_market",
                            "Marktkapitalisierung": "Market_cap",
                            "FK-Kosten_bereinigt": "Debt_costs_interestbearing",
                            "EK-Quote_MKT": "Equity_ratio_market",
                            "FK-Quote_MKT": "Debt_ratio_market",
                            "Aktienanzahl": "Shares_outstanding",
                            "Aktienkurs": "Share_price",
                            "FK-Kosten_unbereinigt": "Debt_costs_total",
                            "Zinsaufwand": "Interest_expense",
                            "FK_zinstragend": "Debt_interestbearing",
                            "Steuersaetze": "Tax_rate",
                            "Marktrendite_Mittelwert": "Market_return_average",
                            "Marktrendite_1Y": "Market_return_annual",
                            "Marktrendite_10Y": "Market_return_10year_rolling",
                            "Investitionsquote": "Investment_ratio",
                            "Umsatzerloese": "Revenue"
                            })

# Inspect column names
wacc.columns

Index(['Company', 'Sector', 'Industry', 'WACC_cMR', 'WACC_15yMR', 'WACC_10yMR',
       'WACC_FS_aT', 'WACC_FS_bT', 'CAPM_cMR', 'CAPM_15yMR', 'CAPM_10yMR',
       'Beta', 'Risk_free_interest', 'Balance_sum', 'Equity_book', 'Debt_book',
       'Equity_ratio_book', 'Debt_ratio_book', 'Total_assets_market',
       'Market_cap', 'Debt_costs_interestbearing', 'Equity_ratio_market',
       'Debt_ratio_market', 'Debt_costs_total', 'Interest_expense',
       'Debt_interestbearing', 'Tax_rate', 'Market_return_average',
       'Market_return_annual', 'Market_return_10year_rolling',
       'Investment_ratio', 'Revenue'],
      dtype='object')

In [ ]:
# Rename index
wacc.index = wacc.index.rename("Year")

# Inspect first rows
wacc.head()

,Company,Sector,Industry,WACC_cMR,WACC_15yMR,WACC_10yMR,WACC_FS_aT,WACC_FS_bT,CAPM_cMR,CAPM_15yMR,CAPM_10yMR,Beta,Risk_free_interest,Balance_sum,Equity_book,Debt_book,Equity_ratio_book,Debt_ratio_book,Total_assets_market,Market_cap,Debt_costs_interestbearing,Equity_ratio_market,Debt_ratio_market,Debt_costs_total,Interest_expense,Debt_interestbearing,Tax_rate,Market_return_average,Market_return_annual,Market_return_10year_rolling,Investment_ratio,Revenue
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0513,0.0472,0.0443,0.070,0.0898,0.0711,0.0585,0.04950,0.50,0.0410,4427.0,1628.0,2799.0,0.367743,0.632257,4154.0,1355.0,0.067074,0.326248,0.673752,0.025473,71.3,1063.0,0.378,0.08,0.073,0.073,0.000887,5860.0
2005.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0530,0.0448,0.0475,0.081,0.1008,0.0716,0.0500,0.05717,0.51,0.0407,5722.0,2684.0,3038.0,0.469067,0.530933,4908.0,1870.0,0.062606,0.380960,0.619040,0.024358,74.0,1182.0,0.337,0.08,0.271,0.091,-0.056687,6636.0
2006.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0618,0.0594,0.0584,0.075,0.0948,0.0729,0.0687,0.06698,0.58,0.0338,8372.0,2828.0,5544.0,0.337793,0.662207,13218.0,7674.0,0.067847,0.580569,0.419431,0.033189,184.0,2712.0,0.314,0.08,0.220,0.086,2.744601,10084.0
2007.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0435,0.0435,0.0432,0.075,0.0948,0.0397,0.0397,0.03925,0.03,0.0378,8314.0,3023.0,5291.0,0.363604,0.636396,15727.0,10436.0,0.074890,0.663573,0.336427,0.032130,170.0,2270.0,0.318,0.08,0.223,0.066,-0.065697,10299.0
2008.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0617,0.0651,0.0537,0.083,0.1028,0.0712,0.0784,0.05391,0.49,0.0423,9519.0,3386.0,6133.0,0.355710,0.644290,11495.0,5362.0,0.075046,0.466453,0.533547,0.033100,203.0,2705.0,0.288,0.08,-0.404,-0.004,0.093666,10799.0


## 1) Data Engineering

### Dealing with feature types

In [ ]:
# Check dtypes
wacc.dtypes

Company                          object
Sector                           object
Industry                         object
WACC_cMR                        float64
WACC_15yMR                      float64
WACC_10yMR                      float64
WACC_FS_aT                      float64
WACC_FS_bT                      float64
CAPM_cMR                        float64
CAPM_15yMR                      float64
CAPM_10yMR                      float64
Beta                            float64
Risk_free_interest              float64
Balance_sum                     float64
Equity_book                     float64
Debt_book                       float64
Equity_ratio_book               float64
Debt_ratio_book                 float64
Total_assets_market             float64
Market_cap                      float64
Debt_costs_interestbearing      float64
Equity_ratio_market             float64
Debt_ratio_market               float64
Debt_costs_total                float64
Interest_expense                float64


In [ ]:
# Transform categorical types into "category"
columns = ["Company", "Sector", "Industry"]

for col in columns:
  wacc[col] = wacc[col].astype("category")

wacc.dtypes

Company                         category
Sector                          category
Industry                        category
WACC_cMR                         float64
WACC_15yMR                       float64
WACC_10yMR                       float64
WACC_FS_aT                       float64
WACC_FS_bT                       float64
CAPM_cMR                         float64
CAPM_15yMR                       float64
CAPM_10yMR                       float64
Beta                             float64
Risk_free_interest               float64
Balance_sum                      float64
Equity_book                      float64
Debt_book                        float64
Equity_ratio_book                float64
Debt_ratio_book                  float64
Total_assets_market              float64
Market_cap                       float64
Debt_costs_interestbearing       float64
Equity_ratio_market              float64
Debt_ratio_market                float64
Debt_costs_total                 float64
Interest_expense

All columns are of type "category" or "float64"

In [ ]:
# We must remove the last row that only contains NaN values
wacc = wacc.iloc[:-1, :]
wacc.head()

,Company,Sector,Industry,WACC_cMR,WACC_15yMR,WACC_10yMR,WACC_FS_aT,WACC_FS_bT,CAPM_cMR,CAPM_15yMR,CAPM_10yMR,Beta,Risk_free_interest,Balance_sum,Equity_book,Debt_book,Equity_ratio_book,Debt_ratio_book,Total_assets_market,Market_cap,Debt_costs_interestbearing,Equity_ratio_market,Debt_ratio_market,Debt_costs_total,Interest_expense,Debt_interestbearing,Tax_rate,Market_return_average,Market_return_annual,Market_return_10year_rolling,Investment_ratio,Revenue
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0513,0.0472,0.0443,0.070,0.0898,0.0711,0.0585,0.04950,0.50,0.0410,4427.0,1628.0,2799.0,0.367743,0.632257,4154.0,1355.0,0.067074,0.326248,0.673752,0.025473,71.3,1063.0,0.378,0.08,0.073,0.073,0.000887,5860.0
2005.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0530,0.0448,0.0475,0.081,0.1008,0.0716,0.0500,0.05717,0.51,0.0407,5722.0,2684.0,3038.0,0.469067,0.530933,4908.0,1870.0,0.062606,0.380960,0.619040,0.024358,74.0,1182.0,0.337,0.08,0.271,0.091,-0.056687,6636.0
2006.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0618,0.0594,0.0584,0.075,0.0948,0.0729,0.0687,0.06698,0.58,0.0338,8372.0,2828.0,5544.0,0.337793,0.662207,13218.0,7674.0,0.067847,0.580569,0.419431,0.033189,184.0,2712.0,0.314,0.08,0.220,0.086,2.744601,10084.0
2007.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0435,0.0435,0.0432,0.075,0.0948,0.0397,0.0397,0.03925,0.03,0.0378,8314.0,3023.0,5291.0,0.363604,0.636396,15727.0,10436.0,0.074890,0.663573,0.336427,0.032130,170.0,2270.0,0.318,0.08,0.223,0.066,-0.065697,10299.0
2008.0,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0617,0.0651,0.0537,0.083,0.1028,0.0712,0.0784,0.05391,0.49,0.0423,9519.0,3386.0,6133.0,0.355710,0.644290,11495.0,5362.0,0.075046,0.466453,0.533547,0.033100,203.0,2705.0,0.288,0.08,-0.404,-0.004,0.093666,10799.0


In [ ]:
# Change index format from float64 to int
wacc.index = wacc.index.astype(int)

# Inspect first rows
wacc.head()

,Company,Sector,Industry,WACC_cMR,WACC_15yMR,WACC_10yMR,WACC_FS_aT,WACC_FS_bT,CAPM_cMR,CAPM_15yMR,CAPM_10yMR,Beta,Risk_free_interest,Balance_sum,Equity_book,Debt_book,Equity_ratio_book,Debt_ratio_book,Total_assets_market,Market_cap,Debt_costs_interestbearing,Equity_ratio_market,Debt_ratio_market,Debt_costs_total,Interest_expense,Debt_interestbearing,Tax_rate,Market_return_average,Market_return_annual,Market_return_10year_rolling,Investment_ratio,Revenue
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0513,0.0472,0.0443,0.070,0.0898,0.0711,0.0585,0.04950,0.50,0.0410,4427.0,1628.0,2799.0,0.367743,0.632257,4154.0,1355.0,0.067074,0.326248,0.673752,0.025473,71.3,1063.0,0.378,0.08,0.073,0.073,0.000887,5860.0
2005,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0530,0.0448,0.0475,0.081,0.1008,0.0716,0.0500,0.05717,0.51,0.0407,5722.0,2684.0,3038.0,0.469067,0.530933,4908.0,1870.0,0.062606,0.380960,0.619040,0.024358,74.0,1182.0,0.337,0.08,0.271,0.091,-0.056687,6636.0
2006,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0618,0.0594,0.0584,0.075,0.0948,0.0729,0.0687,0.06698,0.58,0.0338,8372.0,2828.0,5544.0,0.337793,0.662207,13218.0,7674.0,0.067847,0.580569,0.419431,0.033189,184.0,2712.0,0.314,0.08,0.220,0.086,2.744601,10084.0
2007,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0435,0.0435,0.0432,0.075,0.0948,0.0397,0.0397,0.03925,0.03,0.0378,8314.0,3023.0,5291.0,0.363604,0.636396,15727.0,10436.0,0.074890,0.663573,0.336427,0.032130,170.0,2270.0,0.318,0.08,0.223,0.066,-0.065697,10299.0
2008,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0617,0.0651,0.0537,0.083,0.1028,0.0712,0.0784,0.05391,0.49,0.0423,9519.0,3386.0,6133.0,0.355710,0.644290,11495.0,5362.0,0.075046,0.466453,0.533547,0.033100,203.0,2705.0,0.288,0.08,-0.404,-0.004,0.093666,10799.0


### Dealing with missing values

In [ ]:
# Create Function that gives percentage of missing values per column in df
def check_missings(df):
  for col in df:
    missings = df[col].isna().sum()
    length = len(df[col])
    ratio = missings / length

    print(f"{col}: {round(ratio*100, 2)}%")

In [ ]:
# Apply function
check_missings(wacc)

Company: 0.0%
Sector: 0.0%
Industry: 0.0%
WACC_cMR: 0.0%
WACC_15yMR: 0.0%
WACC_10yMR: 0.0%
WACC_FS_aT: 0.0%
WACC_FS_bT: 0.2%
CAPM_cMR: 0.0%
CAPM_15yMR: 0.0%
CAPM_10yMR: 0.0%
Beta: 0.0%
Risk_free_interest: 0.0%
Balance_sum: 0.0%
Equity_book: 0.0%
Debt_book: 0.0%
Equity_ratio_book: 0.0%
Debt_ratio_book: 0.0%
Total_assets_market: 0.0%
Market_cap: 0.0%
Debt_costs_interestbearing: 0.0%
Equity_ratio_market: 0.0%
Debt_ratio_market: 0.0%
Debt_costs_total: 0.0%
Interest_expense: 0.0%
Debt_interestbearing: 0.0%
Tax_rate: 0.0%
Market_return_average: 0.0%
Market_return_annual: 0.0%
Market_return_10year_rolling: 0.0%
Investment_ratio: 6.86%
Revenue: 3.92%


There are almost every features complete. For our analysis, the WACC after tax are very much important.

In [ ]:
# Convert index values from floats to integers
wacc.index = wacc.index.astype(int)
wacc.head()

,Company,Sector,Industry,WACC_cMR,WACC_15yMR,WACC_10yMR,WACC_FS_aT,WACC_FS_bT,CAPM_cMR,CAPM_15yMR,CAPM_10yMR,Beta,Risk_free_interest,Balance_sum,Equity_book,Debt_book,Equity_ratio_book,Debt_ratio_book,Total_assets_market,Market_cap,Debt_costs_interestbearing,Equity_ratio_market,Debt_ratio_market,Debt_costs_total,Interest_expense,Debt_interestbearing,Tax_rate,Market_return_average,Market_return_annual,Market_return_10year_rolling,Investment_ratio,Revenue
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0513,0.0472,0.0443,0.070,0.0898,0.0711,0.0585,0.04950,0.50,0.0410,4427.0,1628.0,2799.0,0.367743,0.632257,4154.0,1355.0,0.067074,0.326248,0.673752,0.025473,71.3,1063.0,0.378,0.08,0.073,0.073,0.000887,5860.0
2005,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0530,0.0448,0.0475,0.081,0.1008,0.0716,0.0500,0.05717,0.51,0.0407,5722.0,2684.0,3038.0,0.469067,0.530933,4908.0,1870.0,0.062606,0.380960,0.619040,0.024358,74.0,1182.0,0.337,0.08,0.271,0.091,-0.056687,6636.0
2006,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0618,0.0594,0.0584,0.075,0.0948,0.0729,0.0687,0.06698,0.58,0.0338,8372.0,2828.0,5544.0,0.337793,0.662207,13218.0,7674.0,0.067847,0.580569,0.419431,0.033189,184.0,2712.0,0.314,0.08,0.220,0.086,2.744601,10084.0
2007,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0435,0.0435,0.0432,0.075,0.0948,0.0397,0.0397,0.03925,0.03,0.0378,8314.0,3023.0,5291.0,0.363604,0.636396,15727.0,10436.0,0.074890,0.663573,0.336427,0.032130,170.0,2270.0,0.318,0.08,0.223,0.066,-0.065697,10299.0
2008,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0617,0.0651,0.0537,0.083,0.1028,0.0712,0.0784,0.05391,0.49,0.0423,9519.0,3386.0,6133.0,0.355710,0.644290,11495.0,5362.0,0.075046,0.466453,0.533547,0.033100,203.0,2705.0,0.288,0.08,-0.404,-0.004,0.093666,10799.0


### Dealing with misspellings

We want to check if categorical columns have no misspellings in terms of identical declarations of each category.

Therefore we must have a look on every category of each categorical column

**Check "Company"-column**

In [ ]:
# Get list of unique companies
pd.unique(wacc["Company"]).tolist()

['Adidas AG',
 'Allianz SE',
 'BASF SE',
 'Bayer AG',
 'Beiersdorf AG',
 'BMW AG',
 'Commerzbank AG',
 'Continental AG',
 'Daimler AG',
 'Deutsche Börse AG',
 'Deutsche Bank AG',
 'Deutsche Post AG',
 'Deutsche Telekom AG',
 'E.ON SE',
 'Fresenius MedCare AG',
 'Fresenius SE',
 'HeidelbergCement AG',
 'Henkel AG',
 'Infineon AG',
 'Deutsche Lufthansa AG',
 'Linde AG',
 'Merck KGaA',
 'Münchener Rückversicherungsgesellschaft AG',
 ' ProsiebenSat1. Media SE',
 'RWE AG',
 'SAP SE',
 'Siemens AG',
 'ThyssenKrupp AG',
 'Vonovia SE',
 'Volkswagen AG',
 'Covestro AG',
 'Wirecard AG',
 'MTU Aero Engines',
 'DeliveryHero',
 'DeutscheWohnen']

There are 35 unique companies. The number is attributable to the fact that in recent years, the DAX-30 Index has changed its composition.

Technically, we do not have to make any adjustments. However, in order to be precise, we remove the leading space for " ProsiebenSat1. Media SE".

In [ ]:
# Trim the leading space
wacc["Company"] = wacc["Company"].str.strip()

# Check unique values
pd.unique(wacc["Company"]).tolist()

['Adidas AG',
 'Allianz SE',
 'BASF SE',
 'Bayer AG',
 'Beiersdorf AG',
 'BMW AG',
 'Commerzbank AG',
 'Continental AG',
 'Daimler AG',
 'Deutsche Börse AG',
 'Deutsche Bank AG',
 'Deutsche Post AG',
 'Deutsche Telekom AG',
 'E.ON SE',
 'Fresenius MedCare AG',
 'Fresenius SE',
 'HeidelbergCement AG',
 'Henkel AG',
 'Infineon AG',
 'Deutsche Lufthansa AG',
 'Linde AG',
 'Merck KGaA',
 'Münchener Rückversicherungsgesellschaft AG',
 'ProsiebenSat1. Media SE',
 'RWE AG',
 'SAP SE',
 'Siemens AG',
 'ThyssenKrupp AG',
 'Vonovia SE',
 'Volkswagen AG',
 'Covestro AG',
 'Wirecard AG',
 'MTU Aero Engines',
 'DeliveryHero',
 'DeutscheWohnen']

**Check "Sector"-column**

In [ ]:
# Get list for unique "Sector"
pd.unique(wacc["Sector"]).tolist()

['Consumer Cyclical',
 'Financial Services',
 'Basic Materials',
 'Healthcare',
 'Consumer Defensive',
 'Industrials',
 'Communication Services',
 'Utilities',
 'Technology',
 'Real Estate',
 'Real Estate ']

Here we see duplicate entry of "Real Estate". This

In [ ]:
# Trim all leading and trailing spaces
wacc["Sector"] = wacc["Sector"].str.strip()

# Check the unique values
pd.unique(wacc["Sector"]).tolist()

['Consumer Cyclical',
 'Financial Services',
 'Basic Materials',
 'Healthcare',
 'Consumer Defensive',
 'Industrials',
 'Communication Services',
 'Utilities',
 'Technology',
 'Real Estate']

**Check "Industry"-column**

Lastly, the "Industry"-column is the third categorical column of our dataframe. We must check this one as well

In [ ]:
# Get unique categories of "Industry"
pd.unique(wacc["Industry"]).tolist()

['Footwear & Accessories',
 'Insurance',
 'Chemicals',
 'Drug Manufacturer',
 'Household & Personal Products',
 'Auto Manufacturers',
 'Banks',
 'Auto Parts',
 'Financial Data & Stock Exchange',
 'Freight & Logistics',
 'Telecom Services',
 'Utilities',
 'Medical Care Facilities',
 'Building Materials',
 'Semiconductors',
 'Airlines',
 'Specialty Chemicals',
 'Broadcasting',
 'Software',
 'Specialty Industrial Machinery',
 'Metal Fabrication',
 'Real Estate Services',
 'Aerospace and Defense',
 'Internet Retail',
 'Real Estate-Development']

This time, we do no have to make any amendments. We leave it as it is.

## 2) Viz: Wacc-Parameter Analysis

For this part of the exploratory data analysis, we take a closer look on the WACC-parameters:
- Disclosed WACC after tax: `WACC_FS_aT`
- Beta-factor: `Beta`
- Risk-free interest rate: `Risk_free_interest`
- Market returns: `Market_return_annual`, `Market_return_10yrolling`, `Market_return_average`
- Computed WACC after tax using a constant market return: `WACC_cMR`


**Computing average WACC**

In [ ]:
wacc_mean = wacc.groupby(by = wacc.index, axis = 0).mean()
wacc_mean

,WACC_cMR,WACC_15yMR,WACC_10yMR,WACC_FS_aT,WACC_FS_bT,CAPM_cMR,CAPM_15yMR,CAPM_10yMR,Beta,Risk_free_interest,Balance_sum,Equity_book,Debt_book,Equity_ratio_book,Debt_ratio_book,Total_assets_market,Market_cap,Debt_costs_interestbearing,Equity_ratio_market,Debt_ratio_market,Debt_costs_total,Interest_expense,Debt_interestbearing,Tax_rate,Market_return_average,Market_return_annual,Market_return_10year_rolling,Investment_ratio,Revenue
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004,0.050610,0.043327,0.038127,0.074397,0.094183,0.090483,0.069790,0.054974,0.822000,0.0410,120793.666667,12060.686667,108732.890000,0.318949,0.681057,126814.966667,18206.100000,0.046563,0.357507,0.642493,0.023430,1795.785233,60292.866667,0.371533,0.08,0.073,0.073,0.019621,26687.166667
2005,0.052147,0.039100,0.043417,0.074210,0.094007,0.090883,0.055873,0.067489,0.829333,0.0407,131444.871400,13803.858667,117856.479400,0.320596,0.679406,140744.300000,23985.866667,0.047098,0.387515,0.612485,0.022164,2300.294967,73716.500000,0.364933,0.08,0.271,0.091,0.160145,26833.466667
2006,0.054837,0.052397,0.051390,0.074897,0.094693,0.091113,0.084987,0.082439,0.850333,0.0338,116194.331167,15239.280000,134914.984500,0.322759,0.677241,162205.633333,29277.933333,0.053130,0.414897,0.585103,0.024915,3021.100333,90257.033333,0.362533,0.08,0.220,0.086,0.301764,28752.300000
2007,0.055380,0.054970,0.050337,0.075397,0.095193,0.084310,0.083440,0.073163,0.733667,0.0378,178472.515000,16339.763333,162298.585000,0.310593,0.689396,196309.600000,36575.533333,0.052098,0.436818,0.563182,0.024577,3439.512667,94276.966667,0.347733,0.08,0.223,0.066,0.285549,29723.633333
2008,0.049783,0.052847,0.042507,0.078330,0.098127,0.088260,0.099807,0.060794,0.780333,0.0423,186045.614667,14633.871333,171411.710000,0.275501,0.724508,187631.366667,20982.966667,0.050622,0.303501,0.696499,0.023599,2710.999500,74229.966667,0.303500,0.08,-0.404,-0.004,0.022551,31564.900000
2009,0.056553,0.040233,0.022333,0.076960,0.096757,0.095993,0.050070,-0.000260,0.915000,0.0400,151987.570000,15457.366667,136530.236667,0.300981,0.699019,154756.333333,23493.633333,0.053893,0.371545,0.628455,0.027666,1774.125333,67618.900000,0.299167,0.08,0.238,-0.015,0.069520,28810.933333
2010,0.056343,0.045890,0.014800,0.073927,0.093723,0.096983,0.069577,-0.012059,0.938333,0.0327,109473.690000,17787.873333,148847.750000,0.312008,0.687978,170845.033333,26858.933333,0.048884,0.398256,0.601744,0.022858,1555.439000,69094.666667,0.293067,0.08,0.161,0.700,0.061728,31764.866667
2011,0.054170,0.046507,0.043853,0.072437,0.092233,0.097413,0.075403,0.067820,0.948333,0.0278,178681.976667,17160.586667,140566.656667,0.325182,0.674811,156681.133333,23277.200000,0.049237,0.382295,0.617705,0.026873,1632.838667,72085.866667,0.295167,0.08,-0.147,0.013,0.014056,34074.466667
2012,0.057720,0.038667,0.025550,0.072860,0.092657,0.097597,0.047913,0.013656,0.951667,0.0265,176559.206667,17985.446667,156928.193333,0.326531,0.673466,178130.066667,28313.900000,0.047769,0.409257,0.590743,0.020807,1557.487800,41821.366667,0.297233,0.08,0.291,0.102,0.040113,36086.833333


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=2,
                    subplot_titles=("Disclosed WACC compared to computed WACC",
                                    "Disclosed WACC against risk-free interest rate",
                                    "Disclosed WACC against the Beta-factor",
                                    "Disclosed WACC against various market returns"),
                    specs=[[{"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y":True }, {"secondary_y": True}]],
                    vertical_spacing = 0.1, horizontal_spacing = 0.1)


# First Graph: Disclosed WACC vs. Compared WACC ###############################
fig.add_trace(go.Scatter(x = wacc_mean.index, y = wacc_mean["WACC_FS_aT"], 
                         mode = "lines",
                         name = "Disclosed WACC",
                         marker = dict(color = "blue"),
                         showlegend = True),
              row = 1, col = 1, secondary_y = False)

fig.add_trace(go.Scatter(x = wacc_mean.index, y = wacc_mean["WACC_cMR"],
                         mode = "lines",
                         name = "Computed WACC",
                         marker = dict(color = "#74B0CD")), 
              row = 1, col = 1, secondary_y = False)


# Second Graph: Disclosed WACC against risk-free interest rate #####################
fig.add_trace(go.Scatter(x = wacc_mean.index, y = wacc_mean["WACC_FS_aT"], 
                         mode = "lines",
                         name = "Disclosed WACC",
                         marker = dict(color = "blue"),
                         showlegend = False), 
              row = 1, col = 2, secondary_y = False)

fig.add_trace(go.Scatter(x = wacc_mean.index, y = wacc_mean["Risk_free_interest"],
                         mode = "lines",
                         name = "Risk-free interest rate",
                         marker = dict(color = "#B5CD6F")), 
              row = 1, col = 2, secondary_y = False)


# Third Graph: Disclosed WACC against Beta-factor ##################################
fig.add_trace(go.Scatter(x = wacc_mean.index, y = wacc_mean["WACC_FS_aT"], 
                         mode = "lines",
                         name = "Disclosed WACC",
                         marker = dict(color = "blue"),
                         showlegend = False), 
              row = 2, col = 1, secondary_y = False)

fig.add_trace(go.Scatter(x = wacc_mean.index, y = wacc_mean["Beta"],
                         mode = "lines",
                         name = "Beta",
                         marker = dict(color = "#92D369")), 
              row = 2, col = 1, secondary_y = True)


# Fourth Graph: Disclosed WACC against various market returns #######################
fig.add_trace(go.Scatter(x = wacc_mean.index, y = wacc_mean["Market_return_average"],
                         mode = "lines",
                         name = "Average market return",
                         marker = dict(color = "#E1A24A")), 
              row = 2, col = 2, secondary_y = False)

fig.add_trace(go.Scatter(x = wacc_mean.index, y = wacc_mean["Market_return_annual"], 
                         mode = "lines",
                         name = "1-year rolling market return",
                         marker = dict(color = "#DE8344")), 
              row = 2, col = 2, secondary_y = False)

fig.add_trace(go.Scatter(x = wacc_mean.index, y = wacc_mean["Market_return_10year_rolling"],
                         mode = "lines",
                         name = "10-year rolling market return",
                         marker = dict(color = "#DC653F")), 
              row = 2, col = 2, secondary_y = False)

fig.add_trace(go.Scatter(x = wacc_mean.index, y = wacc_mean["WACC_FS_aT"], 
                         mode = "lines+markers",
                         name = "Disclosed WACC",
                         marker = dict(color = "blue"),
                         showlegend = False),             
              row = 2, col = 2, secondary_y = True)


# Figure layout ####################################################################
fig.update_layout(height=700, width=1200, title_text="WACC-Parameter Analysis", font = dict(size = 12, family = "Arial"), template = "seaborn")
fig.update_annotations(font = {"size": 16},
                       hoverlabel_font = {"size": 12})
fig.show()

## 3) Viz: Financing and Investment Analysis



In [ ]:
# Inspect dataframe
wacc.head()

,Company,Sector,Industry,WACC_cMR,WACC_15yMR,WACC_10yMR,WACC_FS_aT,WACC_FS_bT,CAPM_cMR,CAPM_15yMR,CAPM_10yMR,Beta,Risk_free_interest,Balance_sum,Equity_book,Debt_book,Equity_ratio_book,Debt_ratio_book,Total_assets_market,Market_cap,Debt_costs_interestbearing,Equity_ratio_market,Debt_ratio_market,Debt_costs_total,Interest_expense,Debt_interestbearing,Tax_rate,Market_return_average,Market_return_annual,Market_return_10year_rolling,Investment_ratio,Revenue
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0513,0.0472,0.0443,0.070,0.0898,0.0711,0.0585,0.04950,0.50,0.0410,4427.0,1628.0,2799.0,0.367743,0.632257,4154.0,1355.0,0.067074,0.326248,0.673752,0.025473,71.3,1063.0,0.378,0.08,0.073,0.073,0.000887,5860.0
2005,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0530,0.0448,0.0475,0.081,0.1008,0.0716,0.0500,0.05717,0.51,0.0407,5722.0,2684.0,3038.0,0.469067,0.530933,4908.0,1870.0,0.062606,0.380960,0.619040,0.024358,74.0,1182.0,0.337,0.08,0.271,0.091,-0.056687,6636.0
2006,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0618,0.0594,0.0584,0.075,0.0948,0.0729,0.0687,0.06698,0.58,0.0338,8372.0,2828.0,5544.0,0.337793,0.662207,13218.0,7674.0,0.067847,0.580569,0.419431,0.033189,184.0,2712.0,0.314,0.08,0.220,0.086,2.744601,10084.0
2007,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0435,0.0435,0.0432,0.075,0.0948,0.0397,0.0397,0.03925,0.03,0.0378,8314.0,3023.0,5291.0,0.363604,0.636396,15727.0,10436.0,0.074890,0.663573,0.336427,0.032130,170.0,2270.0,0.318,0.08,0.223,0.066,-0.065697,10299.0
2008,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0617,0.0651,0.0537,0.083,0.1028,0.0712,0.0784,0.05391,0.49,0.0423,9519.0,3386.0,6133.0,0.355710,0.644290,11495.0,5362.0,0.075046,0.466453,0.533547,0.033100,203.0,2705.0,0.288,0.08,-0.404,-0.004,0.093666,10799.0


In [ ]:
print(f"Missing values in percent: {round(wacc.isna().sum()/len(wacc), 3)*100}")

Missing values in percent: Company                         0.0
Sector                          0.0
Industry                        0.0
WACC_cMR                        0.0
WACC_15yMR                      0.0
WACC_10yMR                      0.0
WACC_FS_aT                      0.0
WACC_FS_bT                      0.2
CAPM_cMR                        0.0
CAPM_15yMR                      0.0
CAPM_10yMR                      0.0
Beta                            0.0
Risk_free_interest              0.0
Balance_sum                     0.0
Equity_book                     0.0
Debt_book                       0.0
Equity_ratio_book               0.0
Debt_ratio_book                 0.0
Total_assets_market             0.0
Market_cap                      0.0
Debt_costs_interestbearing      0.0
Equity_ratio_market             0.0
Debt_ratio_market               0.0
Debt_costs_total                0.0
Interest_expense                0.0
Debt_interestbearing            0.0
Tax_rate                        0.0
M

We see around 6.9% of the investment ratios are missing. Therefore we must impute the NaN values in this column.
As imputation technique, we use the average investment ratio that can be computed using on 2 approaches:
- using the average investment ratio across all rows
- using the average investment ratio based on the sector

### Using the average investment ratio based on the sector

In [ ]:
# Get the mean investment ratios for each Sector
invest_per_sector = wacc.groupby("Sector", as_index = False)["Investment_ratio"].mean()
# Print new dataframe
invest_per_sector

,Sector,Investment_ratio
0,Basic Materials,0.143179
1,Communication Services,0.135761
2,Consumer Cyclical,0.100304
3,Consumer Defensive,0.090802
4,Financial Services,0.043911
5,Healthcare,0.142164
6,Industrials,0.044184
7,Real Estate,0.311557
8,Technology,0.140534
9,Utilities,0.027075


In [ ]:
# Check if there is a sector that has no investment ratio
len(wacc["Sector"].unique())

10

In total, there are 11 unique sectors. This number matches the rows of the dataframe above.

Impute for each missing value the mean of the corresponding sector

In [ ]:
for index, row in wacc.iterrows():
  if pd.isna(row["Investment_ratio"]):
    # Get sector of company that has no investment ratio
    sec = row["Sector"]
    invest_means = invest_per_sector.loc[invest_per_sector["Sector"] == sec, "Investment_ratio"].values
    row["Investment_ratio"] = invest_means
    #wacc.loc[index, "Investment_ratio"][0] = invest_means
    print(row["Investment_ratio"])



[0.10030429]
[0.04391057]
[0.14317887]
[0.14216376]
[0.09080217]
[0.10030429]
[0.10030429]
[0.10030429]
[0.04391057]
[0.04391057]
[0.04418368]
[0.13576057]
[0.02707527]
[0.14216376]
[0.14216376]
[0.14317887]
[0.09080217]
[0.14053381]
[0.14317887]
[0.14216376]
[0.04391057]
[0.02707527]
[0.14053381]
[0.04418368]
[0.3115569]
[0.10030429]
[0.14317887]
[0.14317887]
[0.14317887]
[0.14053381]
[0.14053381]
[0.04418368]
[0.04418368]
[0.10030429]
[0.3115569]


In [ ]:
# Check if it worked -> it did not
pd.isna(wacc["Investment_ratio"]).sum()

35

### Using the average investment ratio across rows

In [ ]:
# Fill NaN-values with the mean
wacc["Investment_ratio"] = wacc["Investment_ratio"].fillna(wacc["Investment_ratio"].mean())

In [ ]:
# Check how many 
pd.isna(wacc["Investment_ratio"]).sum()

0

Now that we have no missing investment ratios, we might start visualizing the numbers

This analysis targets the financing and investment behavior throughout the years considering capital structures, investment ratios and debt costs in the context of the disclosed WACC.

We want to draw on two fundamental principles of corporate finance:

- A well balanced capital structure is preferred compared to a highly leveraged financing structure. A company that has too much debt (in the range of 70%-90%) suffers high debt costs, which might lead to unnecessarily high WACC. On the opposite, a company that has plenty of equity suffers high equity costs due to the an risk-bearing minimum rate of return. The perfect capital structure might lead to minimal WACC due to fully exploiting the risk-free interest rate while simultaneously remaining the ability to meet its financial obligations

- From a wider economic perspective, it is suggested that companies realistically set their WACC as the minimum rate of return on their investment projects. Therefore, we assess that relation of investment ratios and the disclosed WACC. We aim to see some inverse relations, i.e. the higher the WACC, the lower the investment ratio, the lower the economic value add.


### Create Finance and Investment Analysis Dashboard

In [ ]:
wacc.head()


,Company,Sector,Industry,WACC_cMR,WACC_15yMR,WACC_10yMR,WACC_FS_aT,WACC_FS_bT,CAPM_cMR,CAPM_15yMR,CAPM_10yMR,Beta,Risk_free_interest,Balance_sum,Equity_book,Debt_book,Equity_ratio_book,Debt_ratio_book,Total_assets_market,Market_cap,Debt_costs_interestbearing,Equity_ratio_market,Debt_ratio_market,Debt_costs_total,Interest_expense,Debt_interestbearing,Tax_rate,Market_return_average,Market_return_annual,Market_return_10year_rolling,Investment_ratio,Revenue
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0513,0.0472,0.0443,0.070,0.0898,0.0711,0.0585,0.04950,0.50,0.0410,4427.0,1628.0,2799.0,0.367743,0.632257,4154.0,1355.0,0.067074,0.326248,0.673752,0.025473,71.3,1063.0,0.378,0.08,0.073,0.073,0.000887,5860.0
2005,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0530,0.0448,0.0475,0.081,0.1008,0.0716,0.0500,0.05717,0.51,0.0407,5722.0,2684.0,3038.0,0.469067,0.530933,4908.0,1870.0,0.062606,0.380960,0.619040,0.024358,74.0,1182.0,0.337,0.08,0.271,0.091,-0.056687,6636.0
2006,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0618,0.0594,0.0584,0.075,0.0948,0.0729,0.0687,0.06698,0.58,0.0338,8372.0,2828.0,5544.0,0.337793,0.662207,13218.0,7674.0,0.067847,0.580569,0.419431,0.033189,184.0,2712.0,0.314,0.08,0.220,0.086,2.744601,10084.0
2007,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0435,0.0435,0.0432,0.075,0.0948,0.0397,0.0397,0.03925,0.03,0.0378,8314.0,3023.0,5291.0,0.363604,0.636396,15727.0,10436.0,0.074890,0.663573,0.336427,0.032130,170.0,2270.0,0.318,0.08,0.223,0.066,-0.065697,10299.0
2008,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0617,0.0651,0.0537,0.083,0.1028,0.0712,0.0784,0.05391,0.49,0.0423,9519.0,3386.0,6133.0,0.355710,0.644290,11495.0,5362.0,0.075046,0.466453,0.533547,0.033100,203.0,2705.0,0.288,0.08,-0.404,-0.004,0.093666,10799.0


In [ ]:
# Import packages
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows = 2, cols = 2,
                    horizontal_spacing = 0.1, vertical_spacing = 0.1,
                    subplot_titles = ["Leverage vs. Disclosed WACC",
                                      "Investments vs. Disclosed WACC",
                                      "Investment intensity vs. Debt costs"],
                    specs = [[{"rowspan": 2, "type": "scatter3d"}, {"type": "scatter"}],
                             [None, {}]],
                    column_widths = [0.5, 0.5])

fig.add_trace(go.Scatter3d(z = wacc["Debt_ratio_market"], y = wacc["WACC_FS_aT"], x = wacc.index, 
                           marker = dict(color = "#3E538F", size = 5, opacity = 0.7), mode = "markers", name = "Leverage vs. WACC"), 
              row = 1, col = 1)

fig.add_trace(go.Scatter(x = wacc["Investment_ratio"], y = wacc["WACC_FS_aT"],
                         marker = dict(color = "#5BB88D", opacity = 0.7), mode = "markers", name = "Investment intensity vs. WACC"),
              row = 1, col = 2)

fig.add_trace(go.Scatter(x = wacc["Investment_ratio"], y = wacc["Debt_costs_interestbearing"],
                         marker = dict(color = "#92D369", size = 5, opacity = 0.7), mode = "markers", name = "Investment intensity vs. Debt costs"),
              row = 2, col = 2)

fig.update_layout(height=700, width=1200, title_text="Finance and Investment Analysis", title_font = dict(color = "darkblue", family = "Arial", size = 24), font = dict(size = 14, family = "Arial"), template = "seaborn")
fig.update_annotations(font = {"size": 14})

## 4) Viz: WACC and Business Model Analysis

### Average disclosed WACC per Sector

Create an animation plot (horizontal bar) that shows the average disclosed WACC ("WACC_FS_aT") per Sector across the years 2004 to 2020.

In [ ]:
# Reset index
wacc = wacc.reset_index()
wacc.head()

,Year,Company,Sector,Industry,WACC_cMR,WACC_15yMR,WACC_10yMR,WACC_FS_aT,WACC_FS_bT,CAPM_cMR,CAPM_15yMR,CAPM_10yMR,Beta,Risk_free_interest,Balance_sum,Equity_book,Debt_book,Equity_ratio_book,Debt_ratio_book,Total_assets_market,Market_cap,Debt_costs_interestbearing,Equity_ratio_market,Debt_ratio_market,Debt_costs_total,Interest_expense,Debt_interestbearing,Tax_rate,Market_return_average,Market_return_annual,Market_return_10year_rolling,Investment_ratio,Revenue
0,2004,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0513,0.0472,0.0443,0.070,0.0898,0.0711,0.0585,0.04950,0.50,0.0410,4427.0,1628.0,2799.0,0.367743,0.632257,4154.0,1355.0,0.067074,0.326248,0.673752,0.025473,71.3,1063.0,0.378,0.08,0.073,0.073,0.000887,5860.0
1,2005,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0530,0.0448,0.0475,0.081,0.1008,0.0716,0.0500,0.05717,0.51,0.0407,5722.0,2684.0,3038.0,0.469067,0.530933,4908.0,1870.0,0.062606,0.380960,0.619040,0.024358,74.0,1182.0,0.337,0.08,0.271,0.091,-0.056687,6636.0
2,2006,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0618,0.0594,0.0584,0.075,0.0948,0.0729,0.0687,0.06698,0.58,0.0338,8372.0,2828.0,5544.0,0.337793,0.662207,13218.0,7674.0,0.067847,0.580569,0.419431,0.033189,184.0,2712.0,0.314,0.08,0.220,0.086,2.744601,10084.0
3,2007,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0435,0.0435,0.0432,0.075,0.0948,0.0397,0.0397,0.03925,0.03,0.0378,8314.0,3023.0,5291.0,0.363604,0.636396,15727.0,10436.0,0.074890,0.663573,0.336427,0.032130,170.0,2270.0,0.318,0.08,0.223,0.066,-0.065697,10299.0
4,2008,Adidas AG,Consumer Cyclical,Footwear & Accessories,0.0617,0.0651,0.0537,0.083,0.1028,0.0712,0.0784,0.05391,0.49,0.0423,9519.0,3386.0,6133.0,0.355710,0.644290,11495.0,5362.0,0.075046,0.466453,0.533547,0.033100,203.0,2705.0,0.288,0.08,-0.404,-0.004,0.093666,10799.0


In [ ]:
# Group dataframe by "Year" and "Sector", compute the mean of all columns and subset df using "WACC_FS_aT"
wacc_sec_year = wacc.groupby(["Year", "Sector"]).mean()[["WACC_FS_aT", "Investment_ratio"]]

# Reset index
wacc_sec_year.reset_index(inplace = True)

# Inspect the new df
wacc_sec_year

,Year,Sector,WACC_FS_aT,Investment_ratio
0,2004,Basic Materials,0.077867,0.025291
1,2004,Communication Services,0.076750,0.198576
2,2004,Consumer Cyclical,0.083600,0.034476
3,2004,Consumer Defensive,0.070550,0.276396
4,2004,Financial Services,0.083740,-0.037203
...,...,...,...,...
165,2020,Healthcare,0.059700,0.100065
166,2020,Industrials,0.075067,0.100065
167,2020,Real Estate,0.028600,0.100065
168,2020,Technology,0.089000,0.100065


In [ ]:
# Check missing values
check_missings(wacc_sec_year)

Year: 0.0%
Sector: 0.0%
WACC_FS_aT: 0.0%
Investment_ratio: 0.0%


Now that we have a consistent dataframe we might be able to correctly plot the average disclosed WACC per Sector in an animated dataframe

In [ ]:
# Import package
import plotly.express as px

# Create Figure
fig = px.bar(wacc_sec_year, x = "WACC_FS_aT", y = "Sector", animation_frame = "Year", range_x = [0, 0.1], hover_name = "Sector",
             labels = {"WACC_FS_aT": "Disclosed WACC after Tax", "Sector": "Sectors"}, title = "Average Disclosed WACC for each Sector",
             template = "seaborn", color = "Sector", 
             color_discrete_sequence = ["#020418", "#291735", "#49204A", "#622453", "#9D2958", "#C5384D", "#D74F46", "#E27554", "#E68E69", "#EFCBB0"])
# Define title
fig.update_layout(showlegend = False, title_font = dict(family = "Arial", size = 22))
fig.show()

### Capital Structure Development

This analysis aims to assess the capital structure of DAX-30 companies throughout the years.

Since the capital structure comprises the ratio of equity plus the ratio of debt, we only plot the equity ratio as vertical bars.

We group equity ratios based on the market value and the book value of the same Sectors.

In [ ]:
# Inspect dataframe
wacc.tail()

,Year,Company,Sector,Industry,WACC_cMR,WACC_15yMR,WACC_10yMR,WACC_FS_aT,WACC_FS_bT,CAPM_cMR,CAPM_15yMR,CAPM_10yMR,Beta,Risk_free_interest,Balance_sum,Equity_book,Debt_book,Equity_ratio_book,Debt_ratio_book,Total_assets_market,Market_cap,Debt_costs_interestbearing,Equity_ratio_market,Debt_ratio_market,Debt_costs_total,Interest_expense,Debt_interestbearing,Tax_rate,Market_return_average,Market_return_annual,Market_return_10year_rolling,Investment_ratio,Revenue
505,2019,Wirecard AG,Technology,Software,0.0623,0.0420,0.0509,0.082,0.1018,0.0742,0.049500,0.06033,0.72,0.0046,5855.0,1923.0,3932.0,0.328437,0.671563,22226.0,18292.0,0.009956,0.823000,0.177000,0.004349,17.1,1718.0,0.300,0.08,0.255,0.083,0.100065,NaN
506,2019,MTU Aero Engines,Industrials,Aerospace and Defense,0.0612,0.0414,0.0501,0.088,0.1078,0.0828,0.055144,0.06729,0.81,0.0046,7765.0,2421.0,5344.0,0.311784,0.688216,18536.0,13192.0,0.010394,0.711696,0.288304,0.004491,24.0,2309.0,0.267,0.08,0.255,0.083,0.100065,NaN
507,2020,MTU Aero Engines,Industrials,Aerospace and Defense,0.0605,0.0389,0.0430,0.084,0.1038,0.0868,0.054774,0.06079,0.86,-0.0019,8104.0,2635.0,5469.0,0.325148,0.674852,16776.0,11307.0,0.008053,0.673999,0.326001,0.004023,22.0,2732.0,0.244,0.08,0.035,0.071,0.100065,NaN
508,2020,DeliveryHero,Consumer Cyclical,Internet Retail,0.0820,0.0524,0.0579,0.128,0.1478,0.0940,0.059400,0.06590,0.93,-0.0019,5774.0,1169.0,4605.0,0.797541,0.797541,29926.0,25321.0,0.020374,0.846120,0.153880,0.013051,60.1,2950.0,0.302,0.08,0.035,0.071,0.100065,NaN
509,2020,DeutscheWohnen,Real Estate,Real Estate-Development,0.0280,0.0196,0.0213,0.030,0.0498,0.0466,0.029100,0.03240,0.47,-0.0019,30797.0,13833.0,16964.0,0.550833,0.550833,33612.0,16648.0,0.014888,0.495299,0.504701,0.010121,171.7,11533.0,0.302,0.08,0.035,0.071,0.100065,NaN


In [ ]:
# Subset the wacc-df
wacc_capstr = wacc.groupby(["Year", "Sector"]).mean() [["Equity_ratio_book", "Equity_ratio_market"]]

# Reset index
wacc_capstr = wacc_capstr.reset_index()

# Inspect new dataframe wacc_capstr
wacc_capstr.head()

,Year,Sector,Equity_ratio_book,Equity_ratio_market
0,2004,Basic Materials,0.377280,0.426868
1,2004,Communication Services,0.426600,0.598789
2,2004,Consumer Cyclical,0.253314,0.286678
3,2004,Consumer Defensive,0.364668,0.549917
4,2004,Financial Services,0.282578,0.155886


In [ ]:
# Import package
import plotly.express as px

# Create Figure
fig = px.bar(wacc_capstr, x = "Sector", y = "Equity_ratio_book", animation_frame = "Year", range_y = [0, 1], hover_name = "Sector", orientation = "v",
             labels = {"Equity_ratio_book": "Equity ratio using book value", "Sector": ""}, title = "Equity ratios for each Sector",
             template = "seaborn", color = "Sector",
             color_discrete_sequence = ["#020418", "#291735", "#49204A", "#622453", "#9D2958", "#C5384D", "#D74F46", "#E27554", "#E68E69", "#EFCBB0"])

# Define title
fig.update_layout(showlegend = False, title_font = dict(family = "Arial", size = 22))
fig.show()